<a href="https://colab.research.google.com/github/arundabas1/Indian_Startup_Funding/blob/main/Startup_Funding_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import re

In [2]:
df = pd.read_csv('/content/unclean_startup_funding.csv')

In [3]:
df1 = df.copy()

In [4]:
df1.head(5)

,Unnamed: 0,Date dd/mm/yyyy,Startup Name,Industry Vertical,SubVertical,City Location,Investors Name,InvestmentnType,Amount in USD,Remarks
0,1,09-01-2020,BYJU’S,E-Tech,E-learning,Bengaluru,Tiger Global Management,Private Equity Round,"20,00,00,000",NaN
1,2,13-01-2020,Shuttl,Transportation,App based shuttle service,Gurgaon,Susquehanna Growth Equity,Series C,"80,48,394",NaN
2,3,09-01-2020,Mamaearth,E-commerce,Retailer of baby and toddler products,Bengaluru,Sequoia Capital India,Series B,"1,83,58,860",NaN
3,4,02-01-2020,wealthbucket,FinTech,Online Investment,New Delhi,Vinod Khatumal,Pre-series A,"30,00,000",NaN
4,5,02-01-2020,Fashor,Fashion and Apparel,Embroiled Clothes For Women,Mumbai,Sprout Venture Partners,Seed Round,"18,00,000",NaN


In [5]:
# only 15% values are available in this column.
df1.drop(['Remarks'],axis=1,inplace=True)

In [6]:
df1.rename(columns={
    'Unnamed: 0':'SNo.',
    'Date dd/mm/yyyy':'date',
    'Startup Name':'startup',
    'Industry Vertical':'vertical',
    'SubVertical':'subvertical',
    'City  Location':'city',
    'Investors Name':'investors',
    'InvestmentnType':'round',
    'Amount in USD':'amount'
},inplace=True)

In [7]:
df1.set_index('SNo.',inplace=True)

In [8]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3044 entries, 1 to 3044
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   date         3044 non-null   object
 1   startup      3044 non-null   object
 2   vertical     2873 non-null   object
 3   subvertical  2108 non-null   object
 4   city         2864 non-null   object
 5   investors    3020 non-null   object
 6   round        3040 non-null   object
 7   amount       2084 non-null   object
dtypes: object(8)
memory usage: 214.0+ KB


# Expression to remove unwanted \\\ sequences.

In [9]:
# Step 1: Cleaning function (fixed)
def clean_text(text):
    # If value is NaN, return it as-is
    if pd.isna(text):
        return text

    # Convert to string only if not NaN
    text = str(text)

    # Removes any sequences like (\xNN, \xNN\xNN, etc.) from text
    matches = re.findall(r'(\\x[0-9a-zA-Z]{2})+', text)
    for m in matches:
        text = text.replace(m, "")

    # 2. Remove generic backslash sequences (\s, \n, \t, or even \News)
    text = re.sub(r'\\+', '', text)

    return " ".join(text.split())

In [10]:
# Step 2: Applying it on columns
df1['startup'] = df1['startup'].apply(clean_text)
df1['vertical'] = df1['vertical'].apply(clean_text)
df1['subvertical'] = df1['subvertical'].apply(clean_text)
df1['city'] = df1['city'].apply(clean_text)
df1['investors'] = df1['investors'].apply(clean_text)
df1['round'] = df1['round'].apply(clean_text)

# Cleaning & Converting Amount Column in Crores

In [11]:
# filling null values with 0

df1['amount']=df1['amount'].fillna('0')


# replaced comma and other unwanted characters by blank space
# replaced Undisclosed and its variations with 0
df1['amount']=df1['amount'].str.replace(',','')
df1['amount']=df1['amount'].str.replace('+','0')
df1['amount']=df1['amount'].str.replace('undisclosed','0')
df1['amount']=df1['amount'].str.replace('unknown','0')
df1['amount']=df1['amount'].str.replace('Undisclosed','0')
df1['amount']=df1['amount'].str.replace('N/A','0')

In [12]:
# Step 1: Cleaning function

def clean_amount(value: str):

    # Cleans strings by removing unwanted escape sequences and extra backslashes.

    if pd.isna(value):  # handle NaN safely
        return value

    # Remove hex escape sequences like \xe2\x80\x99, \xc3\xa9 etc.
    value = re.sub(r'(\\x[0-9a-zA-Z]{2})+', '', str(value))

    # Remove leading/trailing/multiple backslashes
    value = value.replace("\\", "")

    # Strip spaces
    value = value.strip()

    return value

In [13]:
# Step 2: Apply cleaning
df1['amount'] = df1['amount'].apply(clean_amount)

In [14]:
# Step 3: we saw it is object type, we have to convert it to float
df1['amount']=df1['amount'].astype('float')

In [15]:
# function to convert Dollars to INR
def to_inr(dollar):
    inr= dollar*85
    return inr/10000000

In [16]:
df1['amount'] = df1['amount'].apply(to_inr)

# Converting Date Column and Creating New Useful Columns

In [17]:
df1['date'] = pd.to_datetime(df1['date'], dayfirst=True,errors='coerce')

In [18]:
# creating new column "year" and "month"
df1['year'] = df1['date'].dt.year
df1['month'] = df1['date'].dt.month
df1.sample(5)

,date,startup,vertical,subvertical,city,investors,round,amount,year,month
SNo.,,,,,,,,,,
2711,2015-06-16,CouponRaja,Online Coupon & comparison platform,NaN,Mumbai,"Mahesh Pratapneni, Pallab Chatterjee & others",Seed Funding,4.25,2015,6
211,2018-07-23,Yulu Bikes,Technology,Bicycle-Sharing Platform,Bengaluru,NaN,Seed/ Angel Funding,0.00,2018,7
837,2017-05-23,TravelSpice,ECommerce,Travel ECommerce portal,Hyderabad,Eagle10 Ventures,Seed Funding,0.00,2017,5
2573,2015-07-01,iDreamCareer,Ed-Tech Platform,NaN,New Delhi,"ah! Ventures, Vivek Joshi, Mohit Satyanand & o...",Seed Funding,0.00,2015,7
2781,2015-05-14,IndianRoots.com,Lifestyle Ecommerce Portal,NaN,Gurgaon,KJS Group,Private Equity,42.50,2015,5


#Problems in Columns

`startup`
1. remove any websites names like
.in, .com, .bio, .pvt ltd, anything else
2. spelling errors like BYJUS and Byjus both are treated as different,
3. remove special characters like  BYJUâ€™S, " , / '
4. inconsistent lower and upper case.
5. row no. 1400 `Letxe2x80s Barter` error
6.

`vertical`
1. fill nan values at last
2. Tele-Shopping / eCommerce , Used Gadgets Buy / Refurbishing
lot of similar words are written
3. similar words have different spellings
eCommerce, E-Commerce, ECommerce

`subvertical`
1. fill nan values at last
2. lower and uppercase issue
3. row no 1555 "nLoans Comparison platformnnnn (adsbygoogle = window.adsbygoogle || []).push({});nn"
4. row no 1805 'Childrenxe2x80s Entertainment Games & Products'


`city`
1. fill nan values at last
2. row no 2869 Pune / US, 2854 - Bangalore / SFO, 2814 - Dallas / Hyderabad, 2730,2677,2514,2504,2469,2451,2321,2316,2270,2257,2245,2231,2225,2175, and many more


3. spelling error of Chennai, US , USA, Gurgaon/Gurugram , Banglore/Bengaluru both are present, Delhi/New Delhi

check for major cities


`investor`
1. fill nan values at last
2. replace this & other with , and replace others with '' space
3. apply cleaning method as well

`round`
1. one column contains multiple synonyms

# Cleaning City Column

In [19]:
# filling null values
df1['city'] = df1['city'].fillna('Unknown')

In [20]:
# before cleaning
df1[df1['city'].str.contains('/')].head()

,date,startup,vertical,subvertical,city,investors,round,amount,year,month
SNo.,,,,,,,,,,
89,2019-06-04,Leegality,Services,Digital Documentation,Mumbai/Bengaluru,Mumbai Angels,Series A,28.90,2019,6
114,2019-02-08,Zilingo,Ecommerce,Fashion & Apparel,India/Singapore,"Sequoia Capital and Temasek Holdings, EDBI, Bu...",Series D,1921.00,2019,2
118,2019-01-04,Vyome Therapeutics Inc.,Health and Wellness,Specialty pharmaceutical,India/US,"Iron Pillar, Perceptive Advisors, Romulus Capi...",Series D,187.00,2019,1
141,2018-11-23,Signzy,Financial Tech,Digital Onboarding Solution,Mumbai/Bengaluru,Stellaris Venture Partners and Kalaari Capital...,Series A,28.90,2018,11
142,2018-11-24,Engineer.ai,Software,AI Platform,India/US,"Lakestar and Jungle Ventures, Softbanks DeepCore.",Series A,250.75,2018,11


In [21]:
# all the cities having multiple values got cleaned and we kept the first name that was there.

df1['city'] = df1['city'].str.split('/', n=1).str[0].str.strip()

In [22]:
# spelling correction
location_map = {
    "Bengaluru": "Bangalore",
    "Delhi": "New Delhi",
    "Gurugram": "Gurgaon",
    "India": "NCR"

}
df1['city'] = df1['city'].replace(location_map)

# Cleaning Vertical Column

In [23]:
df1['vertical'] = df1['vertical'].fillna('Unknown')
df1['vertical'].isna().sum()

np.int64(0)

In [24]:
# This list comprises 70+ % of total data.
vertical_map = {
    "eCommerce": "E Commerce",
    "ECommerce": "E Commerce",
    "E-Commerce": "E Commerce",
    "E-commerce": "E Commerce",
    "Ecommerce": "E Commerce",
    "ecommerce": "E Commerce",
    "eCommerce platform": "E Commerce",
    "Ed-Tech": "Education",
    "Edtech": "Education",
    "Online Education Platform": "Education",
    "EdTech": "Education",
    "FinTech": "Finance",
    "Fin-Tech": "Finance",
    "Health and Wellness": "Healthcare",
    "Logistics Tech": "Logistics",
    "Transportation": "Transport",
    "Food & Beverage": "Food and Beverages",
    "Food & Beverages": "Food and Beverages",
    "Food and Beverage": "Food and Beverages",
    "Online Food Delivery": "Food and Beverages",
    "Food Delivery Platform": "Food and Beverages",
    "Food": "Food and Beverages",
    "Information Technology": "IT"

}
df1['vertical'] = df1['vertical'].replace(vertical_map)

# Cleaning SubVertical Column

In [25]:
df1['subvertical'] = df1['subvertical'].fillna('Unknown')
df1['subvertical'].isna().sum()

np.int64(0)

In [26]:
def clean_subvertical(text):

    if pd.isna(text):
        return text

    text = str(text)

    # 1. Regex-based replace cleanups
    text = re.sub(r'\(adsbygoogle.*?\)\.push\(\{\}\);', '', text, flags=re.DOTALL)
    text = re.sub(r'(\\x[0-9a-zA-Z]{2})+', '', text)
    text = re.sub(r'n{2,}', ' ', text, flags=re.IGNORECASE)

    # 2. String replace cleanups
    text = text.replace("Children S", "Childrens")

    # 3. Uniform formatting
    text = text.lower()
    text = text.strip()
    text = text.title()

    return text


In [27]:
# cleaned and formatted
df1['subvertical'] = df1['subvertical'].apply(clean_subvertical)

# Cleaning Startup


In [28]:
def clean_startup(name):
    # Remove URLs and suffixes
    name = re.sub(r'https?://\S+|\.com\b|\.in\b|\.bio\b|pvt ltd\b', '', name, flags=re.IGNORECASE)

    # Remove all special characters except letters and spaces
    name = re.sub(r"[^a-zA-Z0-9\s]", '', name)

    # Convert to lower case
    name = name.lower().strip()

    # Remove extra spaces
    name = re.sub(r'\s+', ' ', name)

    return name

In [29]:
df1['startup'] = df1['startup'].apply(clean_startup)

In [30]:
df1['startup'] = df1['startup'].str.title()

In [31]:
startup_map = {
    "Ola Cabs": "Ola",
    "Ola Electric": "Ola",
    "Olacabs": "Ola",
    "Oyorooms": "Oyo",
    "Oyo Rooms": "Oyo",
    "Oyofit": "Oyo",
    "Paytm Marketplace": "Paytm",
    "Urbanclap Technologies Pvt Ltd": "Urbanclap",
    "1Mg Healthkartplus": "1Mg",
    "Dailyhunt": "Daily Hunt",
    "Capillary Tech": "Capillary",
    "Lendingkart": "Lending Kart",
    "Little Black Book Delhi": "Little Black Book",
    "Pipabella": "Pipa Bella",
    "Policy Bazaar": "Poicybazaar",
    "Qyk App": "Qyk",
    "Rapido Bike Taxi": "Rapido",
    "Rawpressery": "Raw Pressery",
    "Renewbuy": "Renew Buy",
    "Shubhloans": "Shubh Loans",
    "The Mons Co": "The Moms Co",
    "Toko Innovation Studios": "Toko Innovations",
    "Transerv": "Transerve",
    "Traveltriangle": "Travel Triangle",
    "Urbanladder": "Urban Ladder",
    "Vanitycube": "Vanity Cube",
    "Veritas Finance Ltd": "Veritas Finance",
    "Vogo Automotive": "Vogo",
    "Vogo Automotive Pvt Ltd": "Vogo",
    "Wowexpress": "Wow Express"

}
df1['startup'] = df1['startup'].replace(startup_map)

In [32]:
df1['startup'].nunique()

2280

# Cleaning Round Column

In [33]:
df1['round'] = df1['round'].fillna('Unknown')

In [34]:
round_map = {
    "Seed/ Angel Funding": "Angel Funding",
    "Seed / Angel Funding": "Angel Funding",
    "Seed/Angel Funding": "Angel Funding",
    "Angel / Seed Funding": "Angel Funding",
    "Seed / Angle Funding": "Angel Funding",
    "Angel Round": "Angel Funding",
    "SeednFunding": "Seed Funding",
    "Seed Round": "Seed Funding",
    "Funding Round": "Seed Funding",
    "Seed Funding Round": "Seed Funding",
    "Maiden Round": "Seed Funding",
    "Seed": "Seed Funding",
    "Private Equity Round": "Private Equity",
    "Private Funding": "Private Equity",
    "Private": "Private Equity",
    "PrivateEquity": "Private Equity",
    "PrivatenEquity": "Private Equity",
    "pre-series A": "Pre Series A",
    "Pre-Series A": "Pre Series A",
    "pre-Series A": "Pre Series A",
    "Pre-series A": "Pre Series A"

}
df1['round'] = df1['round'].replace(round_map)

In [35]:
df1['round'].value_counts().head()

,count
round,
Seed Funding,1399
Private Equity,1364
Angel Funding,143
Debt Funding,25
Series A,24


# Cleaning Investors Column

In [36]:
# Fill NaNs as before
df1['investors'] = df1['investors'].fillna('Undisclosed Investors')

# Function to clean individual investor strings
def clean_and_split_investors(investor_str):
    if pd.isna(investor_str):
        return []

    # Normalize delimiters: " and " to "," only if comma present or 'and' present
    if ("," in investor_str or " and " in investor_str):
        investor_str = re.sub(r"\s+and\s+", ",", investor_str)

    # Normalize other delimiters ; / | to comma
    investor_str = re.sub(r"[;/|]", ",", investor_str)

    # Remove role/context in parentheses, e.g. (founder, FreshMenu)
    investor_str = re.sub(r"\([^)]*\)", "", investor_str)

    # Remove & replaced with "and" or comma for consistency
    investor_str = investor_str.replace("&", ",")

    # Remove extra dots or trailing dots
    investor_str = re.sub(r"\.+", ".", investor_str)  # collapse multiple dots to single
    investor_str = re.sub(r"\.$", "", investor_str)   # remove trailing dot

    # Split by comma
    parts = investor_str.split(",")

    # Strip whitespace and remove empty entries
    parts = [p.strip() for p in parts if p.strip()]

    # Remove duplicates within this row, order preserved
    seen = set()
    cleaned_parts = []
    for p in parts:
        p_lower = p.lower()
        if p_lower not in seen:
            cleaned_parts.append(p)
            seen.add(p_lower)

    return cleaned_parts

df1["investors_clean"] = df1["investors"].apply(clean_and_split_investors)

In [37]:
# this code will do magic.


investors_map = {
    "undisclosed investors": "Undisclosed Investors",
    "undisclosed": "Undisclosed Investors",
    "undisclosed investor": "Undisclosed Investors",
    "not disclosed": "Undisclosed Investors",
    "others": "Undisclosed Investors",
    "sequoia india": "Sequoia Capital",
    "sequoia india capital": "Sequoia Capital",
    "sequoia capital india": "Sequoia Capital",
    "tiger global management": "Tiger Global",
    "tiger global management.": "Tiger Global",
    "softbank vision fund": "Softbank",
    "softbank group corp": "Softbank",
    "softbank ventures korea": "Softbank",
    "softbank corp": "Softbank",
    "softbank group": "Softbank",
    "softbank\\xe2\\x80\\x99s deepcore.": "Softbank",
    "softbank": "Softbank",
    "accel partners": "Accel",
    "accel partner": "Accel",
    "accel india": "Accel",
    "accel (formerly known as accel partners)	": "Accel",
    "accel partners & existing investors": "Accel",
    "kalaari capital accelerator program": "Kalaari Capital",
    "you & mr jones": "The Brandtech Group",
    "flipkart logistics pvt. ltd": "Flipkart",
    "blume ventures": "Blume Ventures",
    "saif partners": "SAIF Partners",
    "saif partners & others": "SAIF Partners",
    "indian angel network": "Indian Angel Network",
    "nexus vnture partners": "Nexus Venture Partners",
    "nexus venture partner": "Nexus Venture Partners",
    "idg ventures": "IDG Ventures",
    "hyderabad angels (at startup heroes event)": "Hyderabad Angels",
    "mumbai angels & group of other investors": "Mumbai Angels",
    "group of angel investors": "Group of Angel Investors",
    "cross border angels & others": "Group of Angel Investors",
    "group of angel investors from us/india": "Group of Angel Investors",
    "& others": "Group of Angel Investors",
    "& other": "Group of Angel Investors",
    "group of investors": "Group of Angel Investors",
    "multiple investors": "Group of Angel Investors",
    "500 startups & other hnis": "500 Startups"


}

# Case-insensitive replace inside lists
def apply_investors_map(lst, mapping):
    cleaned_list = []
    for inv in lst:
        key = inv.lower().strip()  # normalize case
        cleaned_list.append(mapping.get(key, inv))  # replace if in map, else keep original
    return cleaned_list

df1["investors_clean"] = df1["investors_clean"].apply(lambda lst: apply_investors_map(lst, investors_map))


In [38]:
# Standardize capitalization for unmapped names: title case
def standardize_capitalization(lst):
    def capitalize_name(name):
        # Simple title case but handle acronyms (all uppercase) by preserving them
        if name.isupper():
            return name
        else:
            return name.title()
    return [capitalize_name(name) for name in lst]

df1["investors_clean"] = df1["investors_clean"].apply(standardize_capitalization)

In [39]:
df1['investors_clean']

,investors_clean
SNo.,
1,[Tiger Global]
2,[Susquehanna Growth Equity]
3,[Sequoia Capital]
4,[Vinod Khatumal]
5,[Sprout Venture Partners]
...,...
3040,[Asia Pacific Internet Group]
3041,[Karsemven Fund]
3042,"[Exfinity Fund, Growx Ventures]"


In [40]:
# Flatten list of all cleaned investors to identify global duplicates
all_investors = [item.lower() for sublist in df1["investors_clean"] for item in sublist]

# Identify duplicates globally
from collections import Counter
investor_counts = Counter(all_investors)

In [41]:
len(all_investors)

6227

In [42]:
# Convert list of investors back to comma-separated string
df1['investors_clean'] = df1['investors_clean'].apply(lambda x: ', '.join(map(str, x)))

In [43]:
df1['investors_clean']

,investors_clean
SNo.,
1,Tiger Global
2,Susquehanna Growth Equity
3,Sequoia Capital
4,Vinod Khatumal
5,Sprout Venture Partners
...,...
3040,Asia Pacific Internet Group
3041,Karsemven Fund
3042,"Exfinity Fund, Growx Ventures"


In [44]:
df1.head()

,date,startup,vertical,subvertical,city,investors,round,amount,year,month,investors_clean
SNo.,,,,,,,,,,,
1,2020-01-09,Byjus,E-Tech,E-Learning,Bangalore,Tiger Global Management,Private Equity,1700.000000,2020,1,Tiger Global
2,2020-01-13,Shuttl,Transport,App Based Shuttle Service,Gurgaon,Susquehanna Growth Equity,Series C,68.411349,2020,1,Susquehanna Growth Equity
3,2020-01-09,Mamaearth,E Commerce,Retailer Of Baby And Toddler Products,Bangalore,Sequoia Capital India,Series B,156.050310,2020,1,Sequoia Capital
4,2020-01-02,Wealthbucket,Finance,Online Investment,New Delhi,Vinod Khatumal,Pre Series A,25.500000,2020,1,Vinod Khatumal
5,2020-01-02,Fashor,Fashion and Apparel,Embroiled Clothes For Women,Mumbai,Sprout Venture Partners,Seed Funding,15.300000,2020,1,Sprout Venture Partners


In [45]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3044 entries, 1 to 3044
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   date             3044 non-null   datetime64[ns]
 1   startup          3044 non-null   object        
 2   vertical         3044 non-null   object        
 3   subvertical      3044 non-null   object        
 4   city             3044 non-null   object        
 5   investors        3044 non-null   object        
 6   round            3044 non-null   object        
 7   amount           3044 non-null   float64       
 8   year             3044 non-null   int32         
 9   month            3044 non-null   int32         
 10  investors_clean  3044 non-null   object        
dtypes: datetime64[ns](1), float64(1), int32(2), object(7)
memory usage: 261.6+ KB


In [46]:
# Explode the list of investors and count unique values
unique_investors_count = df1['investors_clean'].str.split(', ').explode().nunique()
print(f"Number of unique investors: {unique_investors_count}")

Number of unique investors: 3044


In [51]:
df1.to_csv('clean_startup_funding.csv',index=True)